In [1]:
import sys
sys.path.append('C:\My_files\Programming\work')
import pandas as pd
from processing import YouScanProcessing as ysp
import numpy as np
import re
from gspread_pandas import Spread, Client
import gspread_pandas
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [2]:
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('C:/My_files/Programming/work/google_secret.json', scope)
gc = gspread.authorize(credentials)
sht = gc.open('Geely SL Report')

In [3]:
df = pd.read_excel('december.xlsx')

In [4]:
cars = [col for col in df.columns if col.startswith('4_')]
temp_dict = {}
for car in cars:
    temp_dict[car] = {
        'Amount': df[(df[car] == car)]['Дата'].count()
    }
top_cars = pd.DataFrame(temp_dict).transpose().sort_values('Amount', ascending=False)[:3].index.to_list()
top_cars

['4_Atlas', '4_Coolray', '4_Atlas Pro']

In [5]:
def del_tag_num(name):
    return re.sub('\d{1}_', '', name)

In [6]:
last_month_main = [
int(sht.get_worksheet(0).acell('B3').value),
int(sht.get_worksheet(0).acell('B4').value),
int(sht.get_worksheet(0).acell('B5').value),
int(sht.get_worksheet(0).acell('B6').value)]
last_month_main

[14562, 278018, 32535546, 7267]

In [7]:
temp_dict = {
    'Posts': df.shape[0],
    'Engagement': df[['Репосты','Комментарии','Сумма всех реакций']].sum().sum(),
    'OTS': df.groupby('Профиль').agg({'Подписчики': 'mean'}).sum()[0],
    'Reviews': df['1_Отзыв'].count()
}

main = pd.DataFrame(temp_dict, index=['amount']) \
         .transpose().astype(int)
main['R'] = main.amount.apply(ysp.rounding)
main['Change'] = (main.amount / last_month_main -1).round(3)
main

,amount,R,Change
Posts,15312,15.3K,0.052
Engagement,225444,225.4K,-0.189
OTS,44902198,44.9M,0.380
Reviews,7206,7.2K,-0.008


# DAILY COUNT / NSI

In [8]:
daily_count = df.groupby('Дата') \
                .agg({'Время': 'count'}) \
                .rename(columns={'Время': 'Amount'})
daily_count['Average'] = daily_count.Amount \
                                    .mean() \
                                    .round() \
                                    .astype(int)
daily_count

,Amount,Average
Дата,,
01.12.2021,614,494
02.12.2021,429,494
03.12.2021,579,494
04.12.2021,370,494
05.12.2021,460,494
06.12.2021,424,494
07.12.2021,377,494
08.12.2021,591,494
09.12.2021,471,494


In [9]:
nsi = ysp.nsi(df)
nsi.rename(columns={'Негативная': 'Negative', 
                    'Нейтральная': 'Neutral', 
                    'Позитивная': 'Positive', 
                    'Итого': 'Total',}, inplace=True)
nsi

,Negative,Neutral,Positive,Total,NSI
01.12.2021,44,534,36,614,0.86
02.12.2021,27,371,31,429,0.87
03.12.2021,37,511,31,579,0.87
04.12.2021,28,321,21,370,0.85
05.12.2021,36,386,38,460,0.84
06.12.2021,28,367,29,424,0.87
07.12.2021,27,320,30,377,0.86
08.12.2021,27,536,28,591,0.91
09.12.2021,26,411,34,471,0.89
10.12.2021,30,336,29,395,0.85


# NEWS REVIEWS

In [10]:
temp_dict = {}
for car in cars:
    temp_dict[re.sub('\d{1}_','',car)] = {
        'Reviews': df[(df[car] == car)]['1_Отзыв'].count(),
        'News': df[(df[car] == car)]['1_Новость/Статья'].count()
    }
news_reviews = pd.DataFrame(temp_dict).transpose()
news_reviews['Total'] = news_reviews['News'] + news_reviews['Reviews']
news_reviews = news_reviews.sort_values('Total', ascending=False)[:10]
news_reviews

,Reviews,News,Total
Atlas,2242,842,3084
Coolray,1393,661,2054
Atlas Pro,901,890,1791
Tugella,485,420,905
Geely KX11,124,478,602
MK Cross,227,139,366
Emgrand 7,168,80,248
Emgrand X7,160,77,237
Emgrand EC7,197,39,236
Geometry,44,137,181


# TOTAL TONALITY

In [11]:
last_month_tonality = [
int(sht.get_worksheet(0).acell('F3').value),
int(sht.get_worksheet(0).acell('F4').value),
int(sht.get_worksheet(0).acell('F5').value),
int(sht.get_worksheet(0).acell('F6').value)]
last_month_tonality

[1285, 12386, 891, 14562]

In [12]:
total_tonality = nsi.loc['Итого'].to_frame()[:-1].astype(int)
total_tonality['in %'] = (total_tonality['Итого'] / total_tonality['Итого'].iloc[-1]).round(3)
total_tonality.rename(columns={'Итого':'Amount'},inplace=True)
total_tonality['last month'] = last_month_tonality
total_tonality['change'] = total_tonality['Amount'] - total_tonality['last month']
total_tonality['change %'] = (total_tonality['Amount']/total_tonality['last month']-1).round(3)
total_tonality

,Amount,in %,last month,change,change %
Negative,1248,0.082,1285,-37,-0.029
Neutral,13081,0.854,12386,695,0.056
Positive,983,0.064,891,92,0.103
Total,15312,1.000,14562,750,0.052


# REVIEWS TONALITY

In [13]:
temp_dict = {}
temp_dict['Negative'] = df[df['1_Отзыв'] == '1_Отзыв'] \
                    .query('Тональность == "Негативная"') \
                    .agg({'Тональность': 'count'})[0]
temp_dict['Neutral'] = df[df['1_Отзыв'] == '1_Отзыв'] \
                    .query('Тональность == "Нейтральная"') \
                    .agg({'Тональность': 'count'})[0]
temp_dict['Positive'] = df[df['1_Отзыв'] == '1_Отзыв'] \
                    .query('Тональность == "Позитивная"') \
                    .agg({'Тональность': 'count'})[0]

reviews_tonality = pd.DataFrame(temp_dict,index=['Reviews tonality']).transpose()
reviews_tonality['in %'] = (reviews_tonality['Reviews tonality'] / reviews_tonality.sum()[0]).round(3)

reviews_tonality

,Reviews tonality,in %
Negative,1248,0.173
Neutral,4980,0.691
Positive,978,0.136


# SEX

In [14]:
temp_dict = {}
temp_dict['Woman'] = df.query('Демография == "Женщина"').shape[0]
temp_dict['Man'] = df.query('Демография == "Мужчина"').shape[0]
temp_dict['Community'] = df.query('Демография == "Сообщество"').shape[0]

demography = pd.DataFrame(temp_dict,index=['Type']).transpose()
demography['in %'] = (demography['Type'] / demography.sum()[0]).round(3)

demography

,Type,in %
Woman,988,0.089
Man,6338,0.570
Community,3797,0.341


# SOURCE TYPES

In [15]:
source_types = df.groupby('Тип источника') \
             .agg({'Тип источника': 'count'}) \
             .rename(columns={'Тип источника':'count'})
source_types.at['Блог и форумы'] = source_types.loc['Блог'] + source_types.loc['Форум']
source_types = source_types.iloc[np.r_[1:5, 6:7]].sort_index().astype(int)
in_perc = [(f'{source_types.loc[index][0].astype(int)} ({(source_types.loc[index][0] / source_types.sum()[0] * 100).round().astype(int)}%)') for index in source_types.index]
source_types['in %'] = in_perc

source_types

,count,in %
Тип источника,,
Блог и форумы,979,979 (6%)
Мессенджеры,1862,1862 (12%)
Отзывы,573,573 (4%)
СМИ,367,367 (2%)
Соц. сеть,11531,11531 (75%)


# SOURCE

In [16]:
source = df.groupby('Источник') \
             .agg({'Источник': 'count'}) \
             .rename(columns={'Источник': 'scount'}) \
             .sort_values('scount', ascending=False)[:10]
source = source \
        .append(pd.DataFrame([df['Источник'].nunique()], 
                             columns=['scount'])) \
        .rename({0: 'Unique source'})
source['in %'] = (source['scount'] / df['Источник'].count()).round(3)
source

,scount,in %
vk.com,6084,0.397
youtube.com,3251,0.212
telegram.me,2119,0.138
instagram.com,1209,0.079
facebook.com,521,0.034
drive2.ru,362,0.024
maps.yandex.ru,299,0.020
ok.ru,221,0.014
twitter.com,191,0.012
maps.google.com,151,0.010


# CITY

In [17]:
cities = df.groupby('Город').agg({'Город': 'count'}) \
                   .rename(columns={'Город': 'City'}) \
                   .sort_values('City', ascending=False)[:10]
cities

,City
Город,
Москва,1078
Санкт-Петербург,365
Саратов,151
Тюмень,138
Казань,130
Челябинск,105
Нижний Новгород,100
Смоленск,91
Волгоград,88


# AGE

In [18]:
labels = ['<18', '18–24', '25–32', '33–40', '41–45', '46–52', '>52']
bins = [0, 17, 24, 32, 40, 45, 52, 1000]
df['category'] = pd.cut(df['Возраст'], bins=bins, labels=labels)

ages = df.groupby('category') \
         .agg({'Возраст': 'count'}) \
         .sort_values('Возраст',ascending=False)
ages

,Возраст
category,
25–32,568
33–40,553
41–45,352
46–52,179
18–24,147
>52,141
<18,86


# LINKS

In [19]:
top_all_links = df.groupby(['Место публикации','Профиль места публикации']) \
                  .agg({'Дата': 'count'}) \
                  .sort_values('Дата', ascending=False)[:15]

# TONALITY MODELS

In [20]:
tonality_visual = pd.DataFrame()
tonality = pd.DataFrame()
for car in cars:
    temp_df = ysp.tonality(df, [car], car)
    tonality = tonality.append(temp_df) \
                .sort_values('Total', ascending=False)
tonality = ysp.percent(tonality, div_round=3, to_percent=False, 
                calc_columns=['Negative', 'Neutral', 'Positive', 'Total'], 
                percent_columns=['Negative %', 'Neutral %', 'Positive %'],) \
                .rename(index=del_tag_num)

tonality_visual['positive'] = [f'{tonality.Positive.loc[index]} ({(tonality["Positive %"].loc[index] * 100).round().astype(int)}%)' for index in tonality.index]
tonality_visual['neutral'] = [f'{tonality.Neutral.loc[index]} ({(tonality["Neutral %"].loc[index] * 100).round().astype(int)}%)' for index in tonality.index]
tonality_visual['negative'] = [f'{tonality.Negative.loc[index]} ({(tonality["Negative %"].loc[index] * 100).round().astype(int)}%)' for index in tonality.index]
tonality_visual = tonality_visual.iloc[:7].set_axis([ i for i in tonality.iloc[:7].index], axis=0).transpose()

tonality.iloc[np.r_[:7, -1]]

,Negative,Neutral,Positive,Total,Negative %,Neutral %,Positive %
Atlas,266,2970,308,3544,0.075,0.838,0.087
Coolray,250,1987,205,2442,0.102,0.814,0.084
Atlas Pro,106,1895,87,2088,0.051,0.908,0.042
Tugella,58,1185,68,1311,0.044,0.904,0.052
MK Cross,39,862,14,915,0.043,0.942,0.015
Geely KX11,15,565,29,609,0.025,0.928,0.048
ck,9,466,3,478,0.019,0.975,0.006
Total types,862,11674,799,13335,0.065,0.875,0.060


In [21]:
tonality_visual

,Atlas,Coolray,Atlas Pro,Tugella,MK Cross,Geely KX11,ck
positive,308 (9%),205 (8%),87 (4%),68 (5%),14 (2%),29 (5%),3 (1%)
neutral,2970 (84%),1987 (81%),1895 (91%),1185 (90%),862 (94%),565 (93%),466 (98%)
negative,266 (8%),250 (10%),106 (5%),58 (4%),39 (4%),15 (2%),9 (2%)


# OWNERS/BUYERS

In [22]:
temp_dict = {}
owners_visual = pd.DataFrame()
for car in cars:
    own = df[(df[car] == car)]['3_Владелец'].count()
    pick = df[(df[car] == car)]['3_Возможный покупатель'].count()
    unknown = df[(df[car] == car) & (df['3_Владелец'] != '3_Владелец') & \
                 ((df['3_Возможный покупатель'] != '3_Возможный покупатель'))]['3_Владелец'].isna().sum()
    total = sum([own, pick, unknown])
    temp_dict[re.sub('\d{1}_','',car)] = [own, unknown, pick, total]

owners = pd.DataFrame(temp_dict)
owners = owners.rename(index={0:'Owner', 1:'Not defined', 
                              2:'Potential buyer', 3:'Total'}) \
                .transpose().sort_values(by='Total', ascending=False)
owners = ysp.percent(owners, div_round=3, to_percent=False,
                calc_columns=['Owner','Not defined','Potential buyer','Total'], 
                percent_columns=['Owner %','Not defined %','Potential buyer %'])

owners_visual['owner'] = [f'{owners.Owner.loc[index]} ({(owners["Owner %"].loc[index] * 100).round().astype(int)}%)' for index in owners.index]
owners_visual['potential_buyer'] = [f'{owners["Potential buyer"].loc[index]} ({(owners["Potential buyer %"].loc[index] * 100).round().astype(int)}%)' for index in owners.index]
owners_visual['not_defined'] = [f'{owners["Not defined"].loc[index]} ({(owners["Not defined %"].loc[index] * 100).round().astype(int)}%)' for index in owners.index]
owners_visual = owners_visual.iloc[:6].set_axis([ i for i in owners.iloc[:6].index], axis=0).transpose()

owners.iloc[np.r_[:6, -1]]

,Owner,Not defined,Potential buyer,Total,Owner %,Not defined %,Potential buyer %
Atlas,639,2846,59,3544,0.180,0.803,0.017
Coolray,340,2064,38,2442,0.139,0.845,0.016
Atlas Pro,224,1851,13,2088,0.107,0.886,0.006
Tugella,75,1213,23,1311,0.057,0.925,0.018
MK Cross,88,813,14,915,0.096,0.889,0.015
Geely KX11,5,581,23,609,0.008,0.954,0.038
Total types,1582,11516,237,13335,0.119,0.864,0.018


In [23]:
owners_visual

,Atlas,Coolray,Atlas Pro,Tugella,MK Cross,Geely KX11
owner,639 (18%),340 (14%),224 (11%),75 (6%),88 (10%),5 (1%)
potential_buyer,59 (2%),38 (2%),13 (1%),23 (2%),14 (2%),23 (4%)
not_defined,2846 (80%),2064 (84%),1851 (89%),1213 (92%),813 (89%),581 (95%)


# TOP MODELS MAIN

In [24]:
temp_dict = {}
for car in top_cars:
    temp_dict[re.sub('\d{1}_','',car)] = {'Posts': df[df[car] == car].shape[0],
                      'Engagement': df[df[car] == car][['Репосты','Комментарии','Сумма всех реакций']].sum().sum(),
                      'OTS': df[df[car] == car].groupby('Профиль').agg({'Подписчики': 'mean'}).sum()[0]}
    temp_dict[re.sub('\d{1}_','in % ',car)] = {'Posts': (df[df[car] == car].shape[0] / main.iloc[0][0]).round(3),
                      'Engagement': (df[df[car] == car][['Репосты','Комментарии','Сумма всех реакций']].sum().sum() / main.iloc[1][0]).round(3),
                      'OTS': (df[df[car] == car].groupby('Профиль').agg({'Подписчики': 'mean'}).sum()[0] / main.iloc[2][0]).round(3)}
top_main = pd.DataFrame(temp_dict).transpose()
top_main.OTS = top_main.OTS.apply(ysp.rounding)
top_main

,Posts,Engagement,OTS
Atlas,3544.000,53847.000,9.7M
in % Atlas,0.231,0.239,0.216
Coolray,2442.000,50939.000,3.9M
in % Coolray,0.159,0.226,0.086
Atlas Pro,2088.000,32919.000,4.5M
in % Atlas Pro,0.136,0.146,0.1


# TOP MODELS SOURCE

In [25]:
top_source={}
for car in top_cars:
    temp = df[(df[car] == car)].groupby('Источник') \
                           .agg({'Источник': 'count'}) \
                           .rename(columns={'Источник': car}) \
                           .sort_values(car, ascending=False)[:10]
    temp = temp \
            .append(pd.DataFrame([df[(df[car] == car)]['Источник'].nunique()], columns=[car])) \
            .rename({0: 'Unique source'})
    temp['in %'] = (temp[car] / df['Источник'].count()).round(3)
    top_source['top_source_' + car] = temp

top_source

{'top_source_4_Atlas':                 4_Atlas   in %
 vk.com             1267  0.083
 youtube.com        1111  0.073
 telegram.me         364  0.024
 instagram.com       339  0.022
 drive2.ru           121  0.008
 facebook.com         83  0.005
 ok.ru                49  0.003
 ozon.ru              29  0.002
 twitter.com          26  0.002
 maps.yandex.ru       19  0.001
 Unique source        64  0.004,
 'top_source_4_Coolray':                 4_Coolray   in %
 youtube.com           770  0.050
 vk.com                567  0.037
 telegram.me           417  0.027
 instagram.com         310  0.020
 facebook.com           98  0.006
 drive2.ru              95  0.006
 tiktok.com             20  0.001
 zen.yandex.ru          19  0.001
 maps.yandex.ru         19  0.001
 ok.ru                  14  0.001
 Unique source          68  0.004,
 'top_source_4_Atlas Pro':                 4_Atlas Pro   in %
 vk.com                  709  0.046
 youtube.com             405  0.026
 telegram.me             3

# TOP MODELS SOURCE TYPES

In [26]:
top_source_types = {}
for car in top_cars:    
    temp = df[(df[car] == car)].groupby('Тип источника') \
                               .agg({'Тип источника': 'count'}) \
                               .rename(columns={'Тип источника':'amount'})
    temp.at['Блог и форумы'] = temp.loc['Блог'] + temp.loc['Форум']
    temp = temp.iloc[np.r_[1:5, -1]].sort_index().astype(int)
    in_perc = [(f'{temp.loc[index][0].astype(int)} ({(temp.loc[index][0] / temp.sum()[0] * 100).round().astype(int)}%)') for index in temp.index]
    temp['in %'] = in_perc
    top_source_types['top_source_types_' + car] = temp

top_source_types

{'top_source_types_4_Atlas':                amount        in %
 Тип источника                    
 Блог и форумы     196    196 (6%)
 Мессенджеры       340   340 (10%)
 Отзывы             88     88 (2%)
 СМИ                34     34 (1%)
 Соц. сеть        2886  2886 (81%),
 'top_source_types_4_Coolray':                amount        in %
 Тип источника                    
 Блог и форумы     157    157 (6%)
 Мессенджеры       395   395 (16%)
 Отзывы             49     49 (2%)
 СМИ                49     49 (2%)
 Соц. сеть        1792  1792 (73%),
 'top_source_types_4_Atlas Pro':                amount        in %
 Тип источника                    
 Блог и форумы     189    189 (9%)
 Мессенджеры       303   303 (15%)
 Отзывы             16     16 (1%)
 СМИ                58     58 (3%)
 Соц. сеть        1522  1522 (73%)}

# TOP MODELS TAGS TONALITY

In [27]:
models_ft = {}
for car in top_cars:    
    models_ft['models_ft_' + car] = pd.DataFrame({'Negative': [],'Neutral': [],'Positive': [],'Total': []})
    for col in df.columns:
        if str(col).startswith('2_'):
            neg = df[(df[car] == car) & (df['Тональность'] == 'Негативная')][col].count()
            neu = df[(df[car] == car) & (df['Тональность'] == 'Нейтральная')][col].count()
            pos = df[(df[car] == car) & (df['Тональность'] == 'Позитивная')][col].count()
            temp = pd.DataFrame({col: {'Negative': neg, 
                                        'Neutral': neu, 
                                        'Positive': pos, 
                                        'Total': sum([neg, neu, pos])}}).transpose()
            models_ft['models_ft_' + car] = models_ft['models_ft_' + car].append(temp) \
                                                .sort_values('Total', ascending=False) \
                                                .rename(index=del_tag_num).astype(int)

In [28]:
models_ft

{'models_ft_4_Atlas':             Negative  Neutral  Positive  Total
 Service           95     1161        58   1314
 Quality          156      341       148    645
 Comparison        46      375        97    518
 Package           33      391        81    505
 Price             48      261        58    367
 Design             7      112        17    136
 Brand              6       53         2     61
 Suspension         4       31         2     37
 Rust              10       14         1     25,
 'models_ft_4_Coolray':             Negative  Neutral  Positive  Total
 Service          113      609        47    769
 Quality          147      284        92    523
 Package           78      291        30    399
 Price             81      259        49    389
 Comparison        32      284        71    387
 Design            57      121        35    213
 Brand             55       47         5    107
 Suspension        52       12         0     64
 Rust              33       26         0   

# TOP MODELS TOP LINKS

In [29]:
top_links = {}
for car in top_cars:    
    temp = df[(df[car] == car)].groupby(['Место публикации','Профиль места публикации']) \
                               .agg({'Дата': 'count'}) \
                               .sort_values('Дата', ascending=False)[:15]
    top_links['top_links_' + car] = temp
top_links

{'top_links_4_Atlas':                                                                                                  Дата
 Место публикации                             Профиль места публикации                                
 Geely Atlas / Pro Джили Атлас Про Клуб       http://vk.com/club157408979                          346
 Dizzlike Channel                             https://www.youtube.com/channel/UCb_QWRnYLbPF-2...   310
 Рашн Жизнь                                   https://www.youtube.com/channel/UCI4PItwQdWFnh1...   127
 Клубный сервис                               https://www.youtube.com/channel/UCXWNExSw2KO7e0...    98
 Geely Russia                                 http://vk.com/club83363436                            71
 Отзывы и личный опыт на DRIVE2 > Geely Atlas https://www.drive2.ru/experience/geely/g5513          59
 Лада Веста | Lada Vesta FL                   http://vk.com/club71473650                            55
 GEELY CLUB | Джили клуб Geely.su             https:

# EXPORT

In [30]:
c = gspread_pandas.conf.get_config(conf_dir='C:\My_files\Programming\work', file_name='google_secret.json')
spread = Spread('Geely SL Report', config=c)
spread.sheets

[<Worksheet 'Data' id:0>,
 <Worksheet 'Copy of Data' id:45390782>,
 <Worksheet 'Charts' id:895826056>,
 <Worksheet 'Half_year_comparison' id:1124928921>]

In [ ]:
# spread.df_to_sheet(, sheet='Data', start='')

In [31]:
spread.df_to_sheet(main, sheet='Data', start='A2')
spread.df_to_sheet(daily_count, sheet='Data', start='A8')
spread.df_to_sheet(nsi, sheet='Data', start='E8')
spread.df_to_sheet(total_tonality, sheet='Data', start='E2')
spread.df_to_sheet(reviews_tonality, sheet='Data', start='L2')
spread.df_to_sheet(news_reviews, sheet='Data', start='L8')
spread.df_to_sheet(source_types, sheet='Data', start='L20')
spread.df_to_sheet(source, sheet='Data', start='L27')
spread.df_to_sheet(cities, sheet='Data', start='Q2')
spread.df_to_sheet(ages, sheet='Data', start='Q14')

In [32]:
spread.df_to_sheet(tonality.iloc[np.r_[:7, -1]], sheet='Data', start='A62')
spread.df_to_sheet(owners.iloc[np.r_[:6, -1]], sheet='Data', start='A72')
spread.df_to_sheet(owners_visual, sheet='Data', start='J72')
spread.df_to_sheet(tonality_visual, sheet='Data', start='J62')

In [33]:
spread.df_to_sheet(top_all_links, sheet='Data', start='A44')
spread.df_to_sheet(top_links['top_links_4_Atlas'], sheet='Data', start='E44')
spread.df_to_sheet(top_links['top_links_4_Coolray'], sheet='Data', start='I44')
spread.df_to_sheet(top_links['top_links_4_Atlas Pro'], sheet='Data', start='M44')

In [34]:
spread.df_to_sheet(top_main, sheet='Data', start='P24')
spread.df_to_sheet(top_source['top_source_4_Atlas'], sheet='Data', start='U2')
spread.df_to_sheet(top_source['top_source_4_Coolray'], sheet='Data', start='AA2')
spread.df_to_sheet(top_source['top_source_4_Atlas Pro'], sheet='Data', start='AG2')
spread.df_to_sheet(top_source_types['top_source_types_4_Atlas'], sheet='Data', start='U15')
spread.df_to_sheet(top_source_types['top_source_types_4_Coolray'], sheet='Data', start='AA15')
spread.df_to_sheet(top_source_types['top_source_types_4_Atlas Pro'], sheet='Data', start='AG15')
spread.df_to_sheet(models_ft['models_ft_4_Atlas'], sheet='Data', start='U22')
spread.df_to_sheet(models_ft['models_ft_4_Coolray'], sheet='Data', start='AA22')
spread.df_to_sheet(models_ft['models_ft_4_Atlas Pro'], sheet='Data', start='AG22')